In [360]:
import spacy
import pandas as pd
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF
from sklearn.decomposition import TruncatedSVD
import numpy as np
import nltk
from sklearn.decomposition import NMF
nlp = spacy.load('en_core_web_sm')

In [78]:
tweets = pd.read_pickle(r'/Users/philliprichardson/Metis/Module 5/final df.pkl')

In [338]:
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)



In [405]:
stopwords = nltk.corpus.stopwords.words('english')

# Since all tweets are about E3, no real meaning with keeping these words.
updated_stopwords = ['e', 'e3', 'e32021', ' 2021', '2021 ','2021','2022', 'like', 'good', 'game', 'nintendoe3']
stopwords.extend(updated_stopwords)


# keeping 'of' because it's importance to game titles like breath OF the wild.
del stopwords[71]

In [410]:
# Pre-Processing

def tweet_clean(tweets):
    #lowercase tweets
    tweets_clean = tweets.lower()
    #clean RTs
    tweets_clean = re.sub(r'^rt[\s]+', ' ', tweets_clean)
    #remove emojis and other unicode icons
    tweets_clean = emoji_pattern.sub(r' ', tweets_clean)
    #remove links
    tweets_clean = re.sub(r'https?:\/\/.*[\r\n]*', ' ', tweets_clean)
    #clean up hashtags
    tweets_clean = re.sub(r'#', ' ', tweets_clean)
    #remove single numbers
    tweets_clean = re.sub(r' [0-9] ', ' ', tweets_clean)
    #remove '...'
    tweets_clean = tweets_clean.replace('\u2026',' ')
    #Clean up amp which are ampersands, replace with and. Will get removed either way.
    tweets_clean = tweets_clean.replace('amp', ' ')
    #Remove most punctuation, ' is still included.
#     
    #remove stopwords list
    tweets_clean = ' '.join(word for word in tweets_clean.split() if word not in stopwords)

    return tweets_clean


In [411]:
tweets['cleaned'] = tweets.text.apply(lambda x: tweet_clean(x))


In [462]:
lst = []
check = tweets.cleaned.str.split('.')
check

for i in check:
    for j in i:
        if j != '':
            lst.append(j)
        


In [464]:
new_lst = []
for i in lst:
    new_lst.append(' '.join(word.strip(string.punctuation) for word in i.split()))

In [466]:
def topic_generation(text, vectorizer, topic_modeler, n_words = 7):
    doc_word_vectors = vectorizer.fit_transform(text)
    
    # Fit the topic model.
    doc_topic_vectors = topic_modeler.fit_transform(doc_word_vectors)
    
    # Print the topics.
    vocab = vectorizer.get_feature_names()
    for idx, topic in enumerate(topic_modeler.components_):
        # Select the top 15 words in vocab for this topic.
        top_words = [vocab[i].upper() for i in topic.argsort()[:-n_words-1:-1]]
        print(f"Topic {idx}:\n", ", ".join(top_words), "\n")
    
    return doc_topic_vectors

In [467]:
vectorizer = TfidfVectorizer(lowercase=False, max_df = .8, stop_words = updated_stopwords)
vectorizer2 = CountVectorizer(lowercase=False)
nmf = NMF(7)
lsa = TruncatedSVD(7)

### Most generalizable consolidation of topics

In [468]:
topic_generation(tweets.cleaned, vectorizer, nmf, 20)

Topic 0:
 NINTENDO, DIRECT, ANNOUNCED, DAYS, SAVE, SWITCH, SAVED, WINS, TOMORROW, 15, REVEALED, PLEASE, US, 10, NINTENDOSWITCH, 14, PRESENTATION, MARIO, 13, DIREC 

Topic 1:
 XBOXBETHESDA, XBOX20, LOOKS, HALO, FORZA, STARFIELD, REDFALL, HALOINFINITE, INFINITE, OUTER, BATTLEFIELD, WORLDS, TRAILER, HORIZON, AMAZING, COOL, GAMEPLAY, STALKER, COMING, READY 

Topic 2:
 OF, GUARDIANS, GALAXY, SQUAREENIXPRESENTS, BREATH, WILD, LOOKS, ZELDA, SQUARE, LEGEND, SEA, THIEVES, ENIX, BEST, MARVEL, FINAL, TRAILER, SQUAREENIX, FANTASY, LOT 

Topic 3:
 NINTENDODIRECT, METROID, DREAD, NEW, SWITCH, MARIO, BOTW2, ZELDA, SMASH, WARS, ADVANCE, WARIOWARE, BOTW, WILD, BREATH, PARTY, LOOKS, WARIO, COMING, KAZUYA 

Topic 4:
 XBOX, BETHESDA, SHOWCASE, SQUARE, ENIX, PASS, SERIES, COMING, MICROSOFT, EXCLUSIVE, STARFIELD, CONFERENCE, FRIDGE, XBOX20, PC, MINI, SHOW, BEST, GAMES, PLAYSTATION 

Topic 5:
 GO, LET, LETS, SQUAREENIXPRESENTS, SQUARE, ENIX, TIME, FUCKING, SEE, GOOOO, SQUAREENIX, ALRIGHT, GOOOOO, MINUTES, GO

array([[5.20354804e-04, 2.37771372e-04, 1.88289806e-02, ...,
        1.41583733e-03, 0.00000000e+00, 1.11382388e-02],
       [8.07649609e-05, 0.00000000e+00, 2.35995888e-02, ...,
        0.00000000e+00, 0.00000000e+00, 1.93602958e-02],
       [3.82224232e-02, 3.73529986e-04, 4.38515787e-04, ...,
        8.88034863e-04, 2.83908973e-05, 2.69361242e-04],
       ...,
       [0.00000000e+00, 0.00000000e+00, 4.65383686e-02, ...,
        5.37985482e-03, 6.11320346e-03, 1.00907001e-02],
       [0.00000000e+00, 1.01735670e-03, 6.53003881e-03, ...,
        3.95638539e-03, 1.38864177e-04, 8.38532221e-03],
       [0.00000000e+00, 9.57033486e-03, 1.56039943e-02, ...,
        9.84241508e-04, 2.11938990e-03, 1.11600850e-02]])

### Sentence Documents instead of total tweet documents, much clearer topics compared to total tweet level docs

In [472]:
topic_generation(new_lst, vectorizer, nmf, 20)

Topic 0:
 XBOXBETHESDA, XBOX20, HALOINFINITE, HALO, GO, STARFIELD, REDFALL, SHOWCASE, READY, OUTER, FORZAHORIZON5, FORZA, WORLDS, LET, GAMEPASS, XBOXGAMEPASS, BATTLEFIELD2042, BATTLEFIELD, STALKER, XBOXSERIESX 

Topic 1:
 NINTENDODIRECT, METROID, BOTW2, ZELDA, NINTENDOSWITCH, DREAD, SWITCH, SMASH, GO, WARS, ADVANCE, WARIOWARE, SMASHBROSULTIMATE, BOTW, WARIO, MARIO, BREATH, WILD, BREATHOFTHEWILD2, METROIDDREAD 

Topic 2:
 NINTENDO, DIRECT, ANNOUNCED, DAYS, SWITCH, SAVE, TOMORROW, WINS, SAVED, TODAY, 15, PLEASE, REVEALED, METROID, US, YEAR, PRESENTATION, NINTENDOSWITCH, MARIO, DAY 

Topic 3:
 OF, GUARDIANS, GALAXY, BREATH, WILD, DAY, ONE, ZELDA, BEST, SEA, YEAR, LEGEND, LOT, THIEVES, TRAILER, COMING, FAR, PART, FIRST, FINAL 

Topic 4:
 SQUAREENIXPRESENTS, SQUAREENIX, GUARDIANSOFTHEGALAXY, GUARDIANS, SQUARE, GALAXY, ENIX, FANTASY, FINAL, MARVEL, FINALFANTASY, STRANGE, LIFE, AVENGERS, GO, CHAOS, TIME, MOBILE, FALL, LIFEISSTRANGE 

Topic 5:
 XBOX, BETHESDA, SHOWCASE, XBOX20, EXCLUSIVE, CONF

array([[0.00000000e+00, 2.15429252e-04, 6.71520785e-04, ...,
        0.00000000e+00, 1.25976188e-03, 7.99082554e-03],
       [0.00000000e+00, 3.96780104e-05, 3.84980269e-04, ...,
        0.00000000e+00, 1.82196436e-04, 9.18626610e-03],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        3.76964155e-05, 7.62211536e-05, 1.03384519e-02],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.50687476e-03, 1.91553870e-03, 1.01733986e-02],
       [0.00000000e+00, 1.82898895e-04, 4.00970708e-05, ...,
        1.78735031e-03, 2.58839917e-03, 1.10762197e-02],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        6.21732445e-03, 0.00000000e+00, 2.85621594e-02]])

### Too much overlap in topics with LSA and TF-IDF

In [469]:
topic_generation(lst, vectorizer, lsa, 8)

Topic 0:
 XBOXBETHESDA, XBOX20, XBOX, NINTENDODIRECT, OF, SHOWCASE, LOOKS, NINTENDO 

Topic 1:
 NINTENDODIRECT, NINTENDO, OF, DIRECT, METROID, NEW, TODAY, GO 

Topic 2:
 NINTENDO, OF, DIRECT, XBOX, ANNOUNCED, GAMES, TODAY, DAY 

Topic 3:
 OF, SQUAREENIXPRESENTS, XBOX, GAMES, LOOKS, GUARDIANS, GALAXY, DAY 

Topic 4:
 SQUAREENIXPRESENTS, NINTENDO, SQUAREENIX, XBOXBETHESDA, GUARDIANSOFTHEGALAXY, GUARDIANS, DIRECT, FANTASY 

Topic 5:
 XBOX, BETHESDA, SQUAREENIXPRESENTS, SHOWCASE, XBOX20, SHOW, EXCLUSIVE, STARFIELD 

Topic 6:
 LOOKS, UBISOFTFORWARD, IT, WAIT, UBIFORWARD, SEE, HALO, CAN 



array([[ 0.01277396,  0.01920182,  0.05014881, ..., -0.02702974,
        -0.04506005, -0.02514209],
       [ 0.0160591 ,  0.02388116,  0.06728761, ..., -0.03591351,
        -0.0772278 , -0.04139282],
       [ 0.00611214,  0.00696137,  0.01947247, ..., -0.00791121,
         0.00281023,  0.02454588],
       ...,
       [ 0.01812121,  0.0256909 ,  0.08661945, ..., -0.03613535,
        -0.11238749, -0.06272137],
       [ 0.00868485,  0.01216496,  0.02888053, ..., -0.00801999,
        -0.0030024 ,  0.01344137],
       [ 0.01406663,  0.01226202,  0.03927495, ...,  0.01553242,
        -0.02760722,  0.14388929]])

### Less meaningfull words in some of the topics, like 1 and 0 for Count Vectorizer and NMF

In [470]:
topic_generation(lst, vectorizer2, nmf, 8)

Topic 0:
 OF, GALAXY, GUARDIANS, BREATH, WILD, ONE, ZELDA, SEA 

Topic 1:
 NINTENDO, DIRECT, ANNOUNCED, 2021, DAYS, SWITCH, 15, REVEALED 

Topic 2:
 XBOXBETHESDA, XBOX, XBOX20, BETHESDA, SHOWCASE, HALO, STARFIELD, COMING 

Topic 3:
 TODAY, DAY, SEE, CAN, WAIT, 2021, GET, IT 

Topic 4:
 NINTENDODIRECT, METROID, SWITCH, ZELDA, BOTW2, SMASH, MARIO, DREAD 

Topic 5:
 GAMES, VIDEO, SHOWCASE, SHOW, MANY, COMING, XBOX, PLAY 

Topic 6:
 NEW, LOOKS, METROID, TRAILER, GAME, MARIO, GAMEPLAY, COMING 



array([[7.13466760e-02, 1.02786416e-03, 3.23042950e-03, ...,
        4.06343260e-03, 7.97524784e-03, 6.73850144e-03],
       [6.92016797e-02, 1.46783738e-04, 1.14191911e-04, ...,
        6.86925100e-04, 0.00000000e+00, 0.00000000e+00],
       [1.08364945e-04, 0.00000000e+00, 9.85685919e-05, ...,
        1.31271269e-04, 1.91154602e-04, 4.33092013e-04],
       ...,
       [1.37323114e-01, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.15852060e-03, 0.00000000e+00, 2.82826316e-03, ...,
        0.00000000e+00, 4.63046117e-03, 1.15291358e-01],
       [3.27280970e-03, 0.00000000e+00, 2.91654825e-03, ...,
        0.00000000e+00, 0.00000000e+00, 3.57389024e-02]])

### Way too much overlap in topics in Count Vectorizer and LSA

In [373]:
topic_generation(lst, vectorizer2, lsa, 8)

Topic 0:
 OF, NINTENDO, GAMES, XBOX, NEW, XBOXBETHESDA, NINTENDODIRECT, DAY 

Topic 1:
 NINTENDO, DIRECT, ANNOUNCED, NINTENDODIRECT, DAYS, SWITCH, TODAY, METROID 

Topic 2:
 XBOX, XBOXBETHESDA, GAMES, BETHESDA, XBOX20, SHOWCASE, NEW, SHOW 

Topic 3:
 NINTENDODIRECT, GAMES, XBOX, NEW, BETHESDA, METROID, SEE, TODAY 

Topic 4:
 XBOX, BETHESDA, NINTENDO, GAMES, OF, SHOWCASE, SQUARE, ENIX 

Topic 5:
 GAMES, NEW, LOOKS, SEE, VIDEO, XBOXBETHESDA, TODAY, SHOW 

Topic 6:
 NEW, LOOKS, TRAILER, METROID, GAME, XBOX, MARIO, GET 



array([[ 1.02693707, -0.23733615, -0.09642393, ...,  0.00334322,
        -0.01824536,  0.01552171],
       [ 0.99249734, -0.23465604, -0.12953539, ...,  0.03721514,
        -0.05869752,  0.00748261],
       [ 0.0268861 ,  0.00634153,  0.0311503 , ..., -0.01129828,
         0.01983633,  0.01698861],
       ...,
       [ 0.        , -0.        ,  0.        , ..., -0.        ,
        -0.        , -0.        ],
       [ 0.17815069,  0.02843032,  0.17994192, ..., -0.17676539,
         0.34262011,  0.74708299],
       [ 0.14122257, -0.01237671,  0.13420614, ..., -0.13932512,
         0.13023617,  0.3765814 ]])